In [2]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np
import cv2
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.optim import Adam
from time import time, sleep

from utils import *
from custom_pca import custom_pca
from video_loader import VideoLoader
from autoencoders import *

seed = 42

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Comparison of PCA, PCAAE, OneH and TempConv reconstruction errors when encoding 30 sec of gray scale 256x256 video (R25_gray_scaled.mp4)

In [3]:
video_filename = 'data/R25_gray_scaled.mp4'

def train_helper(model_key, base_model, model_args, video, lrs, models, num_epoch_tune=10, num_epoch=50):
    iteration_per_epoch = int(video.duration_frames/video.batch_size)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Using device', device)
    if device.type == 'cuda':
        print(torch.cuda.get_device_name(0))

    losses_tune = []
    for lr in lrs:
        if len(lrs) == 1:
            losses_tune.append(0)
            break
        torch.manual_seed(seed)
        np.random.seed(seed)
        model = base_model(*model_args).to(device)
        optimizer = Adam(model.parameters(), lr=lr)
        for epoch in range(num_epoch_tune):
            epoch_loss = 0
            for frames in video:
                frames = frames.to(device)
                reconstructed = model(frames)
                loss = crit(frames, reconstructed)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
        losses_tune.append(epoch_loss / iteration_per_epoch)
        if len(losses_tune) >= 2 and losses_tune[-1] > losses_tune[-2]:
            break
    lr = lrs[np.argmin(losses_tune)]
    print('Chosen learning rate:', lr)
    torch.manual_seed(seed)
    np.random.seed(seed)
    model = base_model(*model_args).to(device)
    optimizer = Adam(model.parameters(), lr=lr)

    t1 = time()
    losses = []
    for epoch in range(num_epoch):
        epoch_loss = 0
        for frames in video:
            frames = frames.to(device)
            reconstructed = model(frames)
            loss = crit(frames, reconstructed)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        losses.append(epoch_loss / iteration_per_epoch)
        if (epoch+1) % 3 == 0:
            print(f'\t Error at epoch {epoch+1}:', losses[-1])

    model_dict = {
        'model': model.state_dict(),
        'epoch': num_epoch,
        'optimizer': optimizer.state_dict(),
        'lr': lr,
        'losses_tune': losses_tune,
        'lrs': lrs,
        'losses': losses,
        'epoch_time': (time()-t1)/num_epoch
    }
    models[model_key] = model_dict
    return model

def evaluate_model(model, model_key, models, meta):
    reconstructed_train = video_train.reduce_latent(model, trans=False)
    reconstructed_test = video_test.reduce_latent(model, trans=False)
    original_frames_train_ = original_frames_train.numpy().reshape(-1, *video_train.sample_shape)
    original_frames_test_  = original_frames_test.numpy().reshape(-1, *video_test.sample_shape)
    models[key]['train_error'] = reconstruction_error(original_frames_train_, reconstructed_train.numpy())
    models[key]['test_error'] = reconstruction_error(original_frames_test_, reconstructed_test.numpy())
    print('Training error:', models[key]['train_error'])
    print('Testing error:', models[key]['test_error'])
    reconstructed_train_ = reconstructed_train.numpy().reshape(-1, video_train.height, video_train.width)

video_train = VideoLoader(video_filename, duration=30, gray=True)
video_test =  VideoLoader(video_filename, start=30, duration=10, gray=True)
meta = {'w': video_train.width,
        'h': video_train.height,
        'fps': video_train.fps,
        'bs': video_train.batch_size,
        'gray': video_train.gray}
original_frames_train = video_train.get_all_frames()
original_frames_test = video_test.get_all_frames()
num_epoch, num_epoch_tune = 40, 10
models = {}

In [ ]:
############   PCA autoencoder   ############
lrs = [1e-05, 5e-05, 1e-04]
key = '50-PCAAE'
print('\nModel', key)
video_train.sample_shape = video_test.sample_shape = (1, meta['h'], meta['w'])
all_frames = video_train.get_all_frames()
mean, std = torch.mean(all_frames), torch.std(all_frames)
model = train_helper(key, PCAAutoEncoder, ((1, meta['w'], meta['h']), 50, mean, std), video_train,
                           lrs, models, num_epoch_tune=15)
model.to(torch.device('cpu'))
evaluate_model(model, key, models, meta)

############ One hidden layer AE ############
key = '10-OneHAE'
print('\nModel', key)
lrs = [5e-4, 1e-3]
video_train.sample_shape = video_test.sample_shape = (1, meta['h'], meta['w'])
model = train_helper(key, OneHAutoEncoder, ((1, meta['h'], meta['w']), 10), video_train,
                          lrs, models)
model.to(torch.device('cpu'))
evaluate_model(model, key, models, meta)

############   Temporal Conv AE   ############
key = 'TempConvAE'
print('\nModel', key)
video_train.sample_shape = video_test.sample_shape = (1, 16, meta['h'], meta['w'])
model = train_helper(key, TemporalConvAE, (1, 2, 32), video_train, lrs, models)
model.to(torch.device('cpu'))
evaluate_model(model, key, models, meta)

############  10-Temporal Conv AE  ############
key = '10-TempConvAE'
print('\nModel', key)
video_train.sample_shape = video_test.sample_shape = (1, 16, meta['h'], meta['w'])
model = train_helper(key, TemporalConvAE, (1, 2, 32, 10), video_train,
                                   lrs, models)
model.to(torch.device('cpu'))
evaluate_model(model, key, models, meta)


Model 50-PCAAE
Using device cuda
Tesla P100-PCIE-16GB


In [ ]:
models['10-OneHAE']['losses_tune'], #models['10-OneHAE']['']

In [ ]:
losses = [m['losses'] for m in models.values()]
max_epoch = max([m['epoch'] for m in models.values()])
losses.append([8.126883]*max_epoch)
xs = [range(1, m['epoch']+1) for m in models.values()] + [range(1, max_epoch+1)]
labels = [l for l in models.keys()] + ['PCA']

In [ ]:
plot(xs, losses, labels=labels,
    xlabel='Epoch', ylabel='Error', yrange=(0, 70))
plt.savefig('data_generated/report_figures/reconstruction_error_comparison_pca_oneh_tempconv.png')
for k, m in models.items():
    print('Final error for %s:' % k)
    print('\tTraining:', m['train_error'])
    print('\tTesting:', m['test_error'])
    print()

### TempConv, latent dimension comparison. Computed in week5 (testing_colab.ipynb). Use 10 seconds of R25_gray_scaled.mp4 video scaled to 256x256

In [ ]:
with open('data_generated/all_losses_from_colab_2.json', 'r') as fp:
    all_losses = json.load(fp)

ys = [l for _, l in all_losses.values()]
labels = [l for l in all_losses.keys()]

plot(range(1, 41), ys, labels=labels, yrange=(0, 40),
     xlabel='Epoch', ylabel='Loss')
plt.savefig('data_generated/report_figures/latent_dimension_comparison_tempconv.png')

### TempConv, depth and number of channels comparison

In [ ]:
video_filename = 'data/R25_gray_scaled.mp4'
video = VideoLoader(video_filename, duration=10, gray=True, randit=True)
num_epoch, num_epoch_tune = 30, 10
iteration_per_epoch = int(video.duration_frames/video.batch_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device', device)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))

all_losses = {}
lrs = {1: [1e-4, 5e-4, 0.001, 0.005],
       2: [1e-4, 5e-4, 0.001],
       3: [1e-4, 5e-4, 0.001],
       4: [1e-5, 5e-5, 1e-4, 5e-4],
       5: [1e-5, 5e-5, 1e-4, 5e-4]}
for nlayers in range(1, 6):
    for layerchans in [4, 8, 12, 32, 64]:
        total_time_start = time()
        print(f'Model with {nlayers} layer with {layerchans} channels:')

        losses_tune = []
        for lr in lrs[nlayers]:
            torch.manual_seed(seed)
            np.random.seed(seed)
            model = TemporalConvAE_week5(1, nlayers, layerchans).to(device)
            optimizer = Adam(model.parameters(), lr=lr)
            for epoch in range(num_epoch_tune):
                epoch_loss = 0
                for frames in video:
                    frames = frames.view(-1, 1, video.batch_size, video.height, video.width).to(device)
                    reconstructed = model(frames)
                    loss = crit(frames, reconstructed)

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    epoch_loss += loss.item()
            losses_tune.append(epoch_loss / iteration_per_epoch)
        lr = lrs[nlayers][np.argmin(losses_tune)]
        print('Chosen learning rate:', lr)
        torch.manual_seed(seed)
        np.random.seed(seed)
        model = TemporalConvAE_week5(1, nlayers, layerchans).to(device)
        optimizer = Adam(model.parameters(), lr=lr)

        t1 = time()
        losses = []
        for epoch in range(num_epoch):
            epoch_loss = 0
            for frames in video:
                frames = frames.view(-1, 1, video.batch_size, video.height, video.width).to(device)
                reconstructed = model(frames)
                loss = crit(frames, reconstructed)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            losses.append(epoch_loss / iteration_per_epoch)
            if (epoch+1) % 3 == 0:
                print(f'\t Error at epoch {epoch+1}:', losses[-1])
        all_losses[f'{nlayers},{layerchans}'] = ((time()-t1)/num_epoch, losses)
        print('Total time for model:', sec2string(time()-total_time_start))

In [ ]:
all_losses = dict(sorted(all_losses.items()))
all_losses = {f'{l},{c}': all_losses[f'{l},{c}'] for l in [1,2,3,4,5] for c in [4,8,12,32,64]}

ys = [t[1] for t in all_losses.values()]
plot(range(1, num_epoch+1), ys, labels=all_losses.keys(), xlabel='Epoch', ylabel='Error',
     title='Evolution of loss, optimized lr, legend: nlayers,nchannels',
     styles=['C'+str(c)+'-'+s for c in [0,1,2,3,6] for s in ['s','o','^','.','']],
     yrange=(20, 60), figsize=(20, 12))

In [ ]:
for nlayers in range(1, 6):
    for layerchans in [4, 8, 12, 32, 64]:
        print(nlayers, layerchans)
        model = TemporalConvAE_week5(1, nlayers, layerchans).to(device)
        for frames in video:
            frames = frames.view(-1, 1, video.batch_size, video.height, video.width).to(device)
            reconstructed = model(frames)
            break